In [1]:
from sklearn.datasets import fetch_lfw_people
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Only use people with more than 70 images
lfw_people = fetch_lfw_people(data_home=".", min_faces_per_person=70, resize=0.4)
# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape

# for machine learning we use the 2 data directly (as relative pixel
# positions info is ignored by this model)
X = lfw_people.data
n_features = X.shape[1]

# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print("Total dataset size:")
print("#samples: %d" % n_samples)
print("#features: %d" % n_features)
print("#classes: %d" % n_classes)

Total dataset size:
#samples: 1288
#features: 1850
#classes: 7
